<a href="https://colab.research.google.com/github/1nsidewill/News-Simpler/blob/peter's-work/News_Analysis_ver1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [3]:
!ls

 불용어리스트2.txt		     '토픽 모델링.ipynb'       sample.txt
 chromedriver.exe		      KOBERT.ipynb	       Untitled0.ipynb
'colab버전확인,드라이버 연결.ipynb'   News_Analysis.ipynb
'기사 크롤링.ipynb'		      news_df_210205_v02.csv


In [4]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
#뉴스 데이터 불러오기
columns = ['media','date','title','article_original','url']
news_crawled_df = pd.read_csv('./news_df_210205_v02.csv')
news_crawled_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
866,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...
867,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
868,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...
869,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [6]:
news_crawled_df.isna().sum()

media                0
date                 0
title                0
article_original    13
url                  0
dtype: int64

In [7]:
news_crawled_df.dropna(inplace=True)
news_crawled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 858 entries, 0 to 870
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   media             858 non-null    object
 1   date              858 non-null    object
 2   title             858 non-null    object
 3   article_original  858 non-null    object
 4   url               858 non-null    object
dtypes: object(5)
memory usage: 40.2+ KB


In [8]:
news_crawled_df.reset_index(inplace=True)
news_crawled_df.drop('index',inplace=True,axis=1)
news_crawled_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
853,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...
854,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
855,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...
856,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [ ]:
#불용어 import
stopwords2 = pd.read_csv('./불용어리스트2.txt',header=None,encoding='cp949')
stopwords = list(stopwords2[0])
stopwords

In [10]:
#형태소 토크나이징
from konlpy.tag import Okt
okt = Okt()

articles = news_crawled_df.article_original

articles_morphed = []
for article in articles:
    morph = okt.morphs(article)
    
    stopped = ''
    for word in morph:
        if word not in stopwords:
            stopped += word
            stopped += ' '
            
    articles_morphed.append(stopped)

In [ ]:
#형태소 분해된 컬럼 추가
news_crawled_df['article_morphed'] = articles_morphed
#날짜 칼럼 추가
news_crawled_df['date_pd']=pd.to_datetime(news_crawled_df['date'])
news_crawled_df.head()

In [12]:
#원본 date column drop
news_crawled_df.drop(columns='date',axis=1,inplace=True)

In [ ]:
#기사 날짜 14일마다 그룹화
date_grouped = []
date_range = pd.date_range(start='2020-01-01', end='2020-12-31',periods=26)
L = len(date_range)
R = len(news_crawled_df)

for i, date in enumerate(date_range):
    index_group = []
    if i< L-1:
        for j in range(R):
            if date_range[i] < news_crawled_df.date_pd[j] <= date_range[i+1]:
                index_group.append(j)
        date_grouped.append(index_group)

In [60]:
# 그룹별 tfidf 변환
def tfidfy(time_period_no):
    tfidf = TfidfVectorizer(lowercase=False)
    #그룹에 있는 아티클 가져오기
    article_group = []
    for index in date_grouped[time_period_no]:
      article_group.append(news_crawled_df.article_original[index])
    #article_group = [news_crawled_df.article_original[i] for i in date_grouped[time_period_no]]
    
    art_group_tfidf = tfidf.fit_transform(article_group)
    return art_group_tfidf

In [61]:
#date 그룹 내에서 유사도 높은 기사의 index 가져오는 함수
def get_similar_art_index(group_no, art_group_tfidf, group_index):
    # 그룹 안의 한 기사와 나머지 기사간의 similarity 비교
    sim_pair = cosine_similarity(art_group_tfidf[group_index],art_group_tfidf)
    # 인덱스 내림차순으로 소트
    sorted_index = sim_pair.argsort()[:,::-1]
    # 자기자신은 빼기
    sorted_index = sorted_index[:,1:]
    #print(sorted_index)
    
    #유사도가 큰 순으로 유사도 값을 재정렬하되 자기 자신은 제외
    art_sim_value = np.sort(sim_pair.reshape(-1))[::-1]
    art_sim_value = art_sim_value[1:]
    #print(art_sim_value)
    
    #기사 real index의 reference point 구하기
    ref = 0
    for i in range(group_no):
        ref += len(date_grouped[i])
       # print(ref)

    # 유사도가 0.3보다 높은 real 인덱스만 뽑아오기.
    index_high = []
    for idx, value in enumerate(art_sim_value):
        if value>=0.3:
            group_index = sorted_index[0][idx]
            real_index = ref + group_index
            index_high.append(real_index)
            
    return index_high

In [ ]:
#전체 문서에 대해, 날짜 구간 당 유사한 기사의 index를 리스트에 저장한다.
sim_list=[]
for group_no in range(len(date_grouped)):
  for idx in range(len(date_grouped[group_no])):
    art_group_tfidf = tfidfy(group_no)
    sim_list.append(get_similar_art_index(group_no, art_group_tfidf, idx))

#dataframe에 컬럼 추가
news_crawled_df['sim_articles'] = sim_list

In [63]:
#유사한 기사가 3개 이상이면 불러오기
idx_list = []
for i,ls in enumerate(news_crawled_df['sim_articles']):
  if len(ls)>=3:
    idx_list.append(i)
#print(idx_list)
news_crawled_df.iloc[idx_list]

,media,title,article_original,url,article_morphed,date_pd,sim_articles
36,조선일보,방탄소년단 그래미 무대 선다 한국 가수 최초,그래미 어워즈는 미국 대중음악 시상식 중 최고 권위를 지닌 시상식으로 한국 가수가...,https://news.naver.com/main/read.nhn?mode=LSD&...,그래미 어워즈 미국 대중음악 시상식 중 최고 권위 지닌 시상식 한국 가수 공연 하는...,2020-01-24,"[37, 41, 40]"
37,경향신문,방탄소년단 한국 가수 최초 미 그래미 어워드 공연 확정,그룹 방탄소년단BTS이 제62회 ‘그래미 어워드’Grammy Awards에서 공연한...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 BTS 62회 ‘ 그래미 어워드 ’ Grammy Awards 공연 ...,2020-01-24,"[36, 41, 40]"
39,한겨레,BTS 한국 가수 최초 그래미 공연내년엔 후보 목표,래퍼 릴 나스 엑스가 먼저 등장해 자신의 히트곡 ‘올드 타운 로드’를 부르기 시작...,https://news.naver.com/main/read.nhn?mode=LSD&...,래퍼 릴 나스 엑스 먼저 등장 해 히트 곡 ‘ 올드 타운 로드 ’ 부르기 시작 했다...,2020-01-27,"[40, 41, 43]"
40,경향신문,내년엔 그래미 후보될까짧았던 방탄소년단 무대,방탄소년단은 미국 로스앤젤레스 스테이플스 센터에서 열린 제62회 그래미 시상식에서...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 미국 로스앤젤레스 스테이플스 센터 열린 62회 그래미 시상식 래퍼 릴 나...,2020-01-27,"[39, 43, 41, 37, 36]"
41,조선일보,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,방탄소년단은 이날 미국 로스앤젤레스LA 스테이플스 센터에서 열린 제62회 그래미 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 날 미국 로스앤젤레스 LA 스테이플스 센터 열린 62회 그래미 시상식 2...,2020-01-27,"[39, 36, 37, 40, 43]"
...,...,...,...,...,...,...,...
792,조선일보,BTS 빌보드 2년6개월새 다섯번째 1위 비틀스 이후 최단 기록,그룹 방탄소년단BTS의 새 미니앨범 ‘BE’가 미국 빌보드 메인 음반 차트인 ‘빌보...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 BTS 새 미니앨범 ‘ BE ’ 미국 빌보드 메인 음반 차트 인 ‘...,2020-11-30,"[790, 791, 789]"
795,중앙일보,BTS 한국어 곡으로 싱글차트 1위빌보드 62년 새 역사썼다,방탄소년단BTS이 한국어 곡으로 미국 빌보드 싱글 차트 정상에 올랐다. 빌보드는 3...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 BTS 한국어 곡 미국 빌보드 싱글 차트 정상 올랐다 빌보드 30일 현지...,2020-12-01,"[797, 803, 799]"
797,경향신문,방탄소년단 이번엔 한국어 곡으로 빌보드 싱글차트 정상에,그룹 방탄소년단BTS의 신곡 ‘라이프 고스 온’이 미국 빌보드 메인 싱글 차트 정...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 BTS 신곡 ‘ 라이프 고스 온 ’ 미국 빌보드 메인 싱글 차트 정...,2020-12-01,"[803, 799, 795, 794, 798]"
818,조선일보,BTS MAMA 싹쓸이,그룹 ‘방탄소년단BTS’이 아시아 최고 권위의 케이팝 시상식 ‘2020 MAMA엠넷...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 ‘ 방탄소년단 BTS ’ 아시아 최고 권위 케이팝 시상식 ‘ 2020 MAMA...,2020-12-07,"[820, 819, 817]"


In [65]:
#유사기사 비교.
news_crawled_df.article_original[36]

' 그래미 어워즈는 미국 대중음악 시상식 중 최고 권위를 지닌 시상식으로 한국 가수가 공연하는 것은 이번이 처음이다. 방탄소년단은 작년 제61회 그래미 어워즈에 시상자로 참석한 이후 2년 연속 그래미 무대를 밟게 됐다.  그래미 측은 23일이하 현지시간 홈페이지를 통해 방탄소년단이 오는 26일 미국 로스앤젤레스LA 스테이플스 센터에서 열리는 그래미 어워즈 시상식에서 올드 타운 로드 올 스타즈Old Tow Road AllStars 무대에 함께 한다고 밝혔다.  그래미 측은 빌리 레이 사이러스 래퍼 릴 나스 엑스와 함께 방탄소년단 디플로 메이슨 램지 그리고 다른 깜짝 게스트들이 올드 타운 로드 올 스타즈 무대를 꾸민다고 전했다. 그래미는 홈페이지에 게재한 공연·시상자 명단에도 방탄소년단을 추가했다.  그래미를 주관하는 미국레코드예술과학아카데미NARAS에 따르면 올드 타운 로드 올스타즈는 올해 시상식에서 마련되는 두 개의 특별 코너 중 하나다.  릴 나스 엑스의 올드 타운 로드는 빌보드 싱글 차트 핫 100에서 19주간 1위를 차지한 곡으로 많은 뮤지션과의 협업 버전으로 화제를 모았다. 방탄소년단 리더 RM도 지난해 7월 올드 타운 로드에 피처링한 협업 음원 서울 타운 로드Seoul Tow Road’를 릴 나스 엑스와 함께 발표했다.  방탄소년단은 지난해 그래미 어워즈에 시상자로 초청돼 한국 가수로는 처음 참석했다. 당시 베스트 RB 앨범 부문을 시상한 방탄소년단은 다시 돌아오겠다고 밝힌 바 있다. 이들은 올해 그래미상 후보에는 들지는 못했지만 무대를 하게 돼 약속을 지키게 됐다.  그래미 어워즈는 팝계 최고 권위를 지니는 시상식이지만 비영어권 아티스트에게 배타적이라는 비난을 받았다. 그런 만큼 이번 공연은 방탄소년단의 국제적인 위상을 보여준다는 평가다.'

In [66]:
news_crawled_df.article_original[37]

'그룹 방탄소년단BTS이 제62회 ‘그래미 어워드’Grammy Awards에서 공연한다. 팝계 최고 권위를 지닌 그래미 어워드에서 한국 가수가 공연하는 것은 사상 처음이다.  그래미 어워드 측은 23일현지시간 홈페이지를 통해 방탄소년단이 오는 26일 미국 로스앤젤레스LA 스테이플스 센터에서 열리는 그래미 어워드 시상식에 공연자Performer로 출연한다고 공식 발표했다.  그래미 등에 따르면 컨트리 가수 빌리 레이 사이러스 래퍼 릴 나스 엑스와 함께 방탄소년단 디플로 메이슨 램지 그리고 다른 깜짝 게스트들이 ‘올드 타운 로드 올스타즈’Old Tow Road AllStars 협업 무대를 꾸민다.  그래미는 홈페이지에 게재한 공연자 명단에도 방탄소년단을 추가했다. 이로써 방탄소년단은 작년 제61회 그래미 어워드에 시상자로 참석한 데 이어 올해 공연자로서 2년 연속 그래미 무대를 밟게 됐다.  ‘올드 타운 로드 올스타즈’ 공연은 올해 시상식에 마련된 두 개의 특별 코너special segmets 중 하나다. 릴 나스 엑스의 곡 ‘올드 타운 로드’는 빌보드 싱글 차트 ‘핫 100’에서 19주간 1위를 차지한 곡으로 많은 뮤지션과의 협업 버전으로 화제가 됐다.  방탄소년단 리더 RM도 지난해 7월 ‘올드 타운 로드’에 피처링한 협업 음원 ‘서울 타운 로드’Seoul Tow Road를 릴 나스 엑스와 함께 발표한 적이 있다.  방탄소년단이 그래미에서 공연을 펼치기는 처음이다. 방탄소년단은 ‘빌보드 뮤직 어워드’ ‘아메리칸 뮤직 어워드’에 이어 ‘그래미 어워드’까지 미국 3대 음악 시상식에서 모두 공연하는 기록도 쓰게 됐다.  방탄소년단은 지난해 한국 가수 최초로 그래미 어워드에 공식 초청돼 본 시상식 시상자로 무대에 섰다. 이들은 ‘베스트 RB 앨범’ 부문을 시상했는데 당시 무대에 올라 “다시 돌아오겠다”고 지속적인 도전 의지를 나타냈다.  멤버들은 당시 비하인드 영상에서 “내년에는 꼭 저희가 와서 공연하겠다” “내년에는 무대를 한번 해봤으면 좋겠다” 등 그래미에서 공연

할 일: 유사 기사들 클러스터에서 대표 기사 코사인유사도로 뽑기.  
        대표 기사들 summarize 하기.  
        summarize 된 것을 분석하기.